In [1]:
import pybullet as p
import math
import time
from datetime import datetime

In [2]:
physicsClientID = p.connect(p.GUI)

p.getConnectionInfo(physicsClientID)

import pybullet_data

p.setAdditionalSearchPath(pybullet_data.getDataPath())

robot = 1
numJoints = 0

In [3]:
def import_urdf():
    StartPos_plane = [0, 0, 0]
    # StartOrientation_plane = p.getQuaternionFromEuler([0 ,  0, 0])
    StartOrientation_plane = p.getQuaternionFromEuler([0 ,  -0.03491, 0]) # 2 degrees inclined plane
    plane = p.loadURDF("plane.urdf", StartPos_plane, StartOrientation_plane, useFixedBase=1)
    global robot, numJoints
    StartPos = [0, 0, 3.75]
    StartOrientation = p.getQuaternionFromEuler([1.57, 0, 0])
    robot = p.loadURDF("humanoid.urdf", StartPos, StartOrientation, useFixedBase=0)
    position, orientation = p.getBasePositionAndOrientation(robot)
    numBodies = p.getNumBodies()

    numJoints = p.getNumJoints(robot)
    # p.setGravity(0, 0, 0)
    
def joint_details():
    global robot
    for i in range(numJoints):
        if p.getJointInfo(robot, i)[2] == 0: #only details of the revolute joints
            print(p.getJointInfo(robot, i)[0], str(p.getJointInfo(robot, i)[1]).lstrip('b'))

# *** ALTERNATIVE APPROACH *** (not working)

# def ANGLE_limits(jointIndex, bodyID):
#     lowerLimit = p.getJointInfo(bodyID, jointIndex)[9]
#     upperLimit = p.getJointInfo(bodyID, jointIndex)[8]

#     return lowerLimit, upperLimit

# def targetAngle(lowerLimit, upperLimit, timePeriod, t):
#     Angle = (upperLimit - lowerLimit) * abs(math.sin( (2 * math.pi * t) / timePeriod)) + lowerLimit

#     return Angle

# def joint_control_pos_sin(joint_index, timePeriod = 5, bodyID=robot):

#     t=0

#     maxForce = 0.01

#     lowerLimit, upperLimit = ANGLE_limits(joint_index, bodyID)

#     p.setRealTimeSimulation(1)

#     while True:
#         target_Angle = targetAngle(lowerLimit, upperLimit, timePeriod, t)

#         p.setJointMotorControl2(bodyID, joint_index, p.POSITION_CONTROL,  target_Angle, maxForce)

#         try:
#             timeStep = p.getJointState(bodyID, joint_index)[1] / abs((p.getJointState(bodyID, joint_index)[0] - target_Angle))
#         except ZeroDivisionError:
#             timeStep = 1.41423782742495e-14

#         print(timeStep)
        
#         time.sleep(timeStep)

#         t += timeStep


# sin controller functions

def ANGLE_limits(jointIndex, bodyID):
    lowerLimit = p.getJointInfo(bodyID, jointIndex)[9]
    upperLimit = p.getJointInfo(bodyID, jointIndex)[8]

    return lowerLimit, upperLimit


def targetAngle(lowerLimit, upperLimit, timePeriod, t):
    Angle = (upperLimit - lowerLimit) * \
        abs(math.sin((math.pi * t) / timePeriod)) + lowerLimit

    return Angle


def joint_control_pos_sin_2(joint_index, timePeriod=30, bodyID=robot):

    t = 0

    maxForce = 0.02

    lowerLimit, upperLimit = ANGLE_limits(joint_index, bodyID)

    p.setRealTimeSimulation(1)
    i = 0
    while t <= timePeriod:

        # time.sleep(0.01)

        target_Angle = targetAngle(lowerLimit, upperLimit, timePeriod, t)

        current_angle = p.getJointState(bodyID, joint_index)[0]
        print(target_Angle, current_angle)

        before = datetime.timestamp(datetime.now())

        p.setJointMotorControl2(bodyID, joint_index,
                                p.POSITION_CONTROL, target_Angle, maxForce)

        while True:

            time.sleep(1e-04)
            current_angle = float(p.getJointState(bodyID, joint_index)[0])

            if (current_angle >= min(target_Angle-0.001, target_Angle+0.001) and current_angle <= max(target_Angle-0.001, target_Angle+0.001)):
                break

        after = datetime.timestamp(datetime.now())
        dynamic_timestep = after - before
        print(after - before)
        print()
        print(t)
        print()
        # t += 0.25 #if constant time step is required. But this value is based on system perforamnce
        t += dynamic_timestep
        i = 2


In [4]:
import_urdf()

In [5]:
joint_details()

9 'right_hip'
10 'right_knee'
11 'right_ankle'
12 'left_hip'
13 'left_knee'
14 'left_ankle'


In [ ]:
p.getJointState(robot, 9)

In [ ]:
# p.getJointInfo(robot, 9)
p.getJointState(robot, 9)

In [6]:
p.setRealTimeSimulation(1)
p.setGravity(0,0,-10)

In [5]:
p.setRealTimeSimulation(0)

In [7]:
# time.sleep(2)
joint_control_pos_sin_2(10)

0.0 4.0881999541033005e-06
0.014005899429321289

0

-0.004605418634402369 0.00015054039922442032
0.12900304794311523

0.014005899429321289

-0.047022448801875755 -0.0036199147031471166
0.2220020294189453

0.14300894737243652

-0.11999369601549725 -0.046044906547710596
0.24700403213500977

0.36501097679138184

-0.20110505387435723 -0.119025994266261
0.2649710178375244

0.6120150089263916

-0.2879653821941683 -0.2001575560414725
0.26199793815612793

0.876986026763916

-0.3736333657649324 -0.28702473321143224
0.2590029239654541

1.138983964920044

-0.45804584943152915 -0.37267706781301396
0.26299595832824707

1.397986888885498

-0.5434146213152052 -0.45711374537623173
0.2519969940185547

1.6609828472137451

-0.6248273027103767 -0.5424531436299491
0.25

1.9129798412322998

-0.7051650271941411 -0.6238443956537427
0.24800896644592285

2.1629798412323

-0.7843855482538746 -0.7041839764173106
0.2460019588470459

2.4109888076782227

-0.8624424746441626 -0.7834074306102641
0.24599885940551758

2

In [6]:
p.setRealTimeSimulation(0)
p.resetSimulation()
# go to urdf loading cell

In [7]:
import_urdf()

Changes made in urdf to get a slight front tilt in the robot:

test results:
best angle of root,and hips angles combination <~ 5 degrees or 0.08727 radian (for 0 degree floor)(when feet max angle inwards is 0 degree)

sample:-

1.) root joint (fixed): <origin rpy = "0 0 -0.1745">
 # original => <origin rpy = "0 0 0">

2.) right_hip joint (revolute): <origin rpy = "0 0 0.1745">
 # original => <origin rpy = "0 0 0">

3.) left_hip joint (revolute): <origin rpy = "0 0 0.1745">
 # original => <origin rpy = "0 0 0">


for z formation =>

joint 1 (root,hip) +x each
joint 2 (hip,knee) +x each 
joint 3 (knee,ankle) +x each





for stability: increased damping in all joints